<a href="https://colab.research.google.com/github/BeeGassy/Deep_Q-Learning/blob/main/DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms as T
import numpy as np
import matplotlib.pyplot as plt
import gym

In [ ]:
class Agent():
  replay_buffer_list = [] 
  current_transition = ()
  initial_transition = ()
  
  def __init__(self, size):
    self.rb_size = size

  def action_value(input_preprocessed_image):

    pass

  def the_policy():
    pass

  def q_loss():
    pass

  def q_prime():
    pass

  def preprocess_transition():
    pass
  

   

In [ ]:
class DQN():
  DEFAULT_ENV_NAME = “PongNoFrameskip-v4”
  test_env = gym.make(DEFAULT_ENV_NAME)
  episodes = 400
  time_in_episode = 100
  epsilon = 

  def DQN(self, test_env, episodes, time_in_episode, epsilon):
    self.test_env = test_env
    self.episodes = episodes
    self.time_in_episode = time_in_episode
    self.epsilon = epsilon

  def emulator(self):
    state = self.test_env.reset()
    action = self.test_env.action_space.sample()
    observation, reward, _, _ = self.test_env.step(action)
    transition = (state, action, observation, reward)
    Agent.replay_buffer_list.insert(transition)

    return transition

  def random_action():
    action = self.test_env.action_space.sample()
    observation, reward, _, _ = self.test_env.step(action)
    transition = (state, action, observation, reward)
    Agent.replay_buffer_list.insert(transition)

    return transition

  def preprocessing(self, input_observation):
    np_observation = np.transpose(observation, (0, 3, 1, 2))
    copy_observation = np_observation.copy()
    torch_observation = torch.tensor(copy_observation, dtype=torch.float)
    transform = T.Grayscale()
    grey_scaled_observation = transform(torch_observation)
    return grey_scaled_observation

  def stack_frames(self, input_env, stack_count):
    enviroment = FrameStack(input_env, stack_count)
    return enviroment
    
  
  def dqn(self):
    re_buff = replay_buffer()
    initial_transition = emulator()
    state, action, observation, reward = initial_transition

    for e in range(episodes):
      self.test_env = stack_frames(self.test_env, 4)
      grey_scaled_observation = preprocessing(observation)

      for time_step in range(time_in_episode):
        random_action_prob = random.uniform(0.0, 1.0)
        if random_action_prob < epsilon:
          random_action()
        else: 
          action_at_time_step = action_value()
          #input action input emulator and observe reward and the image associated with it
          #set "s_{t+1}" transition and preprocess *idk what is said in the psuedo code
          #store transition in replay buffer
          #sample random minibatch of transitions from D
          # y_j = r_J
          # or
          # the best next option if we are not in a terminal state
          #perform gradient descent step on action value NOT y_j


IndentationError: ignored

In [ ]:
def emulator():
  pass